<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline

In [17]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med','sci.electronics',
              'sci.space','talk.politics.guns','talk.politics.mideast','talk.politics.misc',
              'talk.religion.misc','misc.forsale']

In [18]:
twenty_train=fetch_20newsgroups(subset='train',categories=categories,
                                remove=('footers','headers','quotes'))

In [19]:
pipeline=Pipeline([('vec',CountVectorizer()),('tf_idf',TfidfTransformer()),
             ('sgd',SGDClassifier())])

In [20]:
pipeline.named_steps

{'vec': CountVectorizer(),
 'tf_idf': TfidfTransformer(),
 'sgd': SGDClassifier()}

In [25]:
parameter={'vec__ngram_range':[(1,2),(2,3)],'tf_idf__norm':('l1','l2'),
           'tf_idf__use_idf':(True,False),'sgd__alpha':(1e-1, 1e-3),
           'sgd__penalty':('l1','l2'),'sgd__max_iter':[10,15] }

In [26]:
parameter_search=GridSearchCV(estimator=pipeline,param_grid=parameter,cv=5,n_jobs=-1)

In [27]:
parameter_search.estimator.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vec', 'tf_idf', 'sgd', 'vec__analyzer', 'vec__binary', 'vec__decode_error', 'vec__dtype', 'vec__encoding', 'vec__input', 'vec__lowercase', 'vec__max_df', 'vec__max_features', 'vec__min_df', 'vec__ngram_range', 'vec__preprocessor', 'vec__stop_words', 'vec__strip_accents', 'vec__token_pattern', 'vec__tokenizer', 'vec__vocabulary', 'tf_idf__norm', 'tf_idf__smooth_idf', 'tf_idf__sublinear_tf', 'tf_idf__use_idf', 'sgd__alpha', 'sgd__average', 'sgd__class_weight', 'sgd__early_stopping', 'sgd__epsilon', 'sgd__eta0', 'sgd__fit_intercept', 'sgd__l1_ratio', 'sgd__learning_rate', 'sgd__loss', 'sgd__max_iter', 'sgd__n_iter_no_change', 'sgd__n_jobs', 'sgd__penalty', 'sgd__power_t', 'sgd__random_state', 'sgd__shuffle', 'sgd__tol', 'sgd__validation_fraction', 'sgd__verbose', 'sgd__warm_start'])

In [28]:
parameter_search.fit(twenty_train.data[:500],twenty_train.target[:500])

C:\anaconda\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vec', CountVectorizer()),
                                       ('tf_idf', TfidfTransformer()),
                                       ('sgd', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'sgd__alpha': (0.1, 0.001), 'sgd__max_iter': [10, 15],
                         'sgd__penalty': ('l1', 'l2'),
                         'tf_idf__norm': ('l1', 'l2'),
                         'tf_idf__use_idf': (True, False),
                         'vec__ngram_range': [(1, 2), (2, 3)]})

The  best_score_ and  store the best mean score and the best_params_ attributes corresponding to that score

In [29]:
parameter_search.best_score_

0.5479999999999999

In [30]:
parameter_search.best_params_

{'sgd__alpha': 0.001,
 'sgd__max_iter': 15,
 'sgd__penalty': 'l2',
 'tf_idf__norm': 'l2',
 'tf_idf__use_idf': True,
 'vec__ngram_range': (1, 2)}

In [31]:
parameter_search.best_estimator_

Pipeline(steps=[('vec', CountVectorizer(ngram_range=(1, 2))),
                ('tf_idf', TfidfTransformer()),
                ('sgd', SGDClassifier(alpha=0.001, max_iter=15))])

In [33]:
docs_new = ['God is good','who is your creator', 'OpenGL on the GPU is fast',
            'I am selling my car','i will pray tonight']
predict=parameter_search.predict(docs_new)
for doc, pred in zip(docs_new,predict):
    print('{}           =========>  {}'.format(doc,twenty_train.target_names[pred]))

God is good           =========>  soc.religion.christian
who is your creator           =========>  alt.atheism
OpenGL on the GPU is fast           =========>  comp.graphics
I am selling my car           =========>  misc.forsale
i will pray tonight           =========>  soc.religion.christian


In [34]:
clf=Pipeline(steps=[('vec', CountVectorizer(ngram_range=(1, 2))),
                ('tf_idf', TfidfTransformer()),
                ('sgd', SGDClassifier(alpha=0.001))])